In [7]:

!pip install --upgrade langchain langchain-community transformers sentencepiece beautifulsoup4 requests --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [8]:

from langchain_community.llms import HuggingFacePipeline
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from transformers import pipeline
import requests
from bs4 import BeautifulSoup


In [9]:

summarizer_pipe = pipeline("summarization", model="facebook/bart-large-cnn")
llm = HuggingFacePipeline(pipeline=summarizer_pipe)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-9-8f70d28a0c22>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=summarizer_pipe)


In [10]:

def fetch_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    article = ' '.join([p.get_text() for p in paragraphs])
    return article[:3000]


In [11]:

def summarize_news(url):
    print("🔗 Fetching article from:", url)
    text = fetch_article_text(url)
    print("\n📰 Original Article Preview:\n", text[:500], "...\n")

    doc = [Document(page_content=text)]
    chain = load_summarize_chain(llm, chain_type="stuff")
    summary = chain.invoke(doc)

    print("✨ Summary:\n", summary)


In [12]:

url = "https://indianexpress.com/article/business/rbi-mpc-meeting-april-2025-live-updates-repo-rate-9933276/?ref=hometop_hp"  # Replace with any news article
summarize_news(url)


🔗 Fetching article from: https://indianexpress.com/article/business/rbi-mpc-meeting-april-2025-live-updates-repo-rate-9933276/?ref=hometop_hp

📰 Original Article Preview:
 RBI MPC Meeting April 2025 Live Updates: The Reserve Bank of India’s Monetary Policy Committee may announce its decision today, Wednesday, at 10 am, with a likely cut in repo rates expected. This is the first RBI MPC meeting of financial year 2025-26, and a 25 basis point cut in the policy repo rate is expected. This comes just after US President Donald Trump’s reciprocal tariffs came into force, including a 104 per cent levy on China, keeping global markets on edge. Alongside the MPC statement, ...



<ipython-input-11-c6bc61080908>:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chain.run(doc)


✨ Summary:
 The Reserve Bank of India’s Monetary Policy Committee may announce its decision today, Wednesday, at 10 am, with a likely cut in repo rates expected. This is the first RBI MPC meeting of financial year 2025-26, and a 25 basis point cut in the policy repo rate is expected. In February, the MPC cut the repo rate by 25 basis points to 6.25 per cent, the first reduction since May 2020.
